In [25]:
import pandas as pd 
import numpy as np 

In [26]:
df = pd.read_csv("proceessed_data.csv")

In [27]:
df.sample(5)

,text,sentiment
16415,chang thay anh online gi ca,0
10226,that sounds cool and youre paying even better,1
23274,oh those messages probably not,0
1969,where are you going paris,0
5033,ooh which two books did you buy,0


In [28]:
import pandas as pd


# Shuffle the DataFrame
df = df.sample(frac=1).reset_index(drop=True)




In [29]:
def fill_null_text(df, fill_text="hello"):
    # Find the first index where 'text' is null
    null_index = df[df['text'].isnull()].index[0]  # Get the first null index
    
    # Replace the null value at the found index with the provided text
    df.at[null_index, 'text'] = fill_text
    
    # Verify the change by printing the updated value
    print(f"Filled index {null_index} with text: {df['text'][null_index]}")
    

fill_null_text(df)

Filled index 24112 with text: hello


In [30]:
df.isnull().sum()

text         0
sentiment    0
dtype: int64

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf=TfidfVectorizer(strip_accents=None,lowercase=False,preprocessor=None,use_idf=True,norm='l2',smooth_idf=True)
y=df.sentiment.values
x=tfidf.fit_transform(df.text)

In [32]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(x,y,random_state=4,test_size=0.5,shuffle=False)

In [33]:
from sklearn.linear_model import LogisticRegressionCV
clf=LogisticRegressionCV(cv=6,scoring='accuracy',random_state=0,n_jobs=-1,verbose=3,max_iter=500).fit(X_train,y_train)
y_pred = clf.predict(X_test)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   6 | elapsed:  2.0min remaining:  1.0min
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  2.0min finished


In [34]:
from sklearn import metrics

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.7382743716984153


In [41]:
import pickle
pickle.dump(clf,open('clf_loggi_reg.pkl','wb'))
pickle.dump(tfidf,open('tfidf.pkl','wb'))

In [35]:
import re

def preprocessing(text):
    
    # Check if text is a string
    if isinstance(text, str):
        # Remove HTML tags
        clean = re.compile('<.*?>')
        text = re.sub(clean, '', text)  # Correctly assign the cleaned text
        
        # Convert text to lowercase
        text = text.lower()
        
        # Remove non-alphabetic characters (keep spaces)
        cleaned_text = re.sub(r'[^a-zA-Z\s]', '', text)
        
        return cleaned_text  # Return the final cleaned text
    else:
        return text  # Return the original text if it's not a string


In [36]:
def prediction(comment):
    preprocessed_comment = preprocessing(comment)
    comment_list = [preprocessed_comment]  # Wrap the preprocessed comment in a list
    comment_vector = tfidf.transform(comment_list)
    prediction = clf.predict(comment_vector)[0]
    return prediction



prediction = prediction('i hate  you ')

In [37]:
if prediction == 0:
    print("neutral comment")
elif prediction == 1:
    print("positive comment")
elif prediction == 2:
    print("negative comment")
else:
    print("suicide comment")

    

negative comment


In [38]:
# Import necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Redefine the Naive Bayes model
nb_model = MultinomialNB()

# Train the Naive Bayes model on the cleaned data
nb_model.fit(X_train, y_train)

# Evaluate the model with predictions
y_pred_cleaned = nb_model.predict(X_test)

# Calculate accuracy, classification report, and confusion matrix
accuracy_cleaned = accuracy_score(y_test, y_pred_cleaned)
classification_rep_cleaned = classification_report(y_test, y_pred_cleaned)
conf_matrix_cleaned = confusion_matrix(y_test, y_pred_cleaned)

accuracy_cleaned, classification_rep_cleaned, conf_matrix_cleaned


(0.503281575156075,
 '              precision    recall  f1-score   support\n\n           0       0.54      0.46      0.50      5628\n           1       0.83      0.34      0.48      4315\n           2       0.83      0.11      0.20      3860\n           3       0.42      1.00      0.59      4938\n\n    accuracy                           0.50     18741\n   macro avg       0.66      0.48      0.44     18741\nweighted avg       0.64      0.50      0.46     18741\n',
 array([[2592,  235,   71, 2730],
        [1205, 1473,   18, 1619],
        [ 991,   60,  439, 2370],
        [   8,    1,    1, 4928]], dtype=int64))

In [39]:
y_pred2 = nb_model.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred2))

Accuracy: 0.503281575156075


In [40]:
# Import necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Step 4: Model Training with XGBoost
xgb_model = XGBClassifier()
xgb_model.fit(X_train, y_train)

# Step 5: Model Evaluation
y_pred = xgb_model.predict(X_test)

# Accuracy and other metrics
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", classification_rep)
print("Confusion Matrix:\n", conf_matrix)


Accuracy: 0.7412091137079131
Classification Report:
               precision    recall  f1-score   support

           0       0.58      0.80      0.68      5628
           1       0.79      0.65      0.72      4315
           2       0.71      0.50      0.59      3860
           3       0.98      0.94      0.96      4938

    accuracy                           0.74     18741
   macro avg       0.77      0.72      0.73     18741
weighted avg       0.76      0.74      0.74     18741

Confusion Matrix:
 [[4511  523  547   47]
 [1325 2816  155   19]
 [1709  164 1943   44]
 [ 171   44  102 4621]]


In [42]:
import pickle
pickle.dump(xgb_model,open('xgb_model.pkl','wb'))


In [43]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from sklearn.metrics import accuracy_score, classification_report

# Step 1: Data Preprocessing
data_cleaned = df.dropna(subset=['text'])
X = df['text']
y = df['sentiment']

# Step 2: Tokenization and Padding
max_words = 5000
max_len = 100

tokenizer = Tokenizer(num_words=max_words, lower=True)
tokenizer.fit_on_texts(X.values)
X_tokenized = tokenizer.texts_to_sequences(X.values)
X_padded = pad_sequences(X_tokenized, maxlen=max_len)

# Step 3: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)

# Step 4: Build LSTM Model
model = Sequential()
model.add(Embedding(max_words, 100, input_length=max_len))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(4, activation='softmax'))   # Assuming 3 sentiment classes

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Step 5: Training the model
batch_size = 50
epochs = 4
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test), verbose=2)

# Step 6: Evaluate Model
y_pred = np.argmax(model.predict(X_test), axis=1)

# Accuracy and classification report
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", classification_rep)


Epoch 1/4
600/600 - 200s - loss: 0.7420 - accuracy: 0.6683 - val_loss: 0.5911 - val_accuracy: 0.7591 - 200s/epoch - 333ms/step
Epoch 2/4
600/600 - 215s - loss: 0.5308 - accuracy: 0.7879 - val_loss: 0.5494 - val_accuracy: 0.7750 - 215s/epoch - 359ms/step
Epoch 3/4
600/600 - 204s - loss: 0.4698 - accuracy: 0.8141 - val_loss: 0.5528 - val_accuracy: 0.7794 - 204s/epoch - 339ms/step
Epoch 4/4
600/600 - 208s - loss: 0.4365 - accuracy: 0.8298 - val_loss: 0.5821 - val_accuracy: 0.7698 - 208s/epoch - 347ms/step
235/235 [==============================] - 5s 19ms/step
Accuracy: 0.7697745764972656
Classification Report:
               precision    recall  f1-score   support

           0       0.70      0.62      0.66      2234
           1       0.75      0.78      0.77      1720
           2       0.65      0.71      0.68      1538
           3       0.95      0.97      0.96      2005

    accuracy                           0.77      7497
   macro avg       0.76      0.77      0.77      7497
wei

In [44]:
import pickle

# Step 1: Save the trained LSTM model
model.save('lstm_sentiment_model.h5')  # This saves the Keras model in HDF5 format

# Step 2: Save the tokenizer using pickle
with open('tokenizer.pkl', 'wb') as file:
    pickle.dump(tokenizer, file)

print("Model and tokenizer saved successfully.")


Model and tokenizer saved successfully.


C:\Users\TRUST\anaconda3\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [53]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
import pickle

# Step 1: Load the saved LSTM model and tokenizer
model = load_model('lstm_sentiment_model.h5')

with open('tokenizer.pkl', 'rb') as file:
    tokenizer = pickle.load(file)

# Step 2: Preprocess the input comment (assuming a simple preprocessing function)


# Step 3: Function for predicting sentiment using the LSTM model
def prediction(comment):
    # Preprocess the input comment
    preprocessed_comment = preprocessing(comment)
    
    # Tokenize and pad the comment
    comment_sequence = tokenizer.texts_to_sequences([preprocessed_comment])
    comment_padded = pad_sequences(comment_sequence, maxlen=100)  # Use the same maxlen used during training
    
    # Make a prediction using the LSTM model
    prediction = np.argmax(model.predict(comment_padded), axis=1)[0]
    
    return prediction

# Step 4: Predict sentiment and print results
input_comment = 'I hated this movie. Hated hated hated hated hated this movie. Hated it." He went on to call it "a bad film - one of the worst movies ever made.'
pred = prediction(input_comment)

if pred == 0:
    print("neutral comment")
elif pred == 1:
    print("positive comment")
elif pred == 2:
    print("negative comment")
else:
    print("suicide comment")


1/1 [==============================] - 0s 198ms/step
negative comment
